<a href="https://colab.research.google.com/github/zahra-zarrabi/RNN/blob/main/joon_del.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten,Dropout,TimeDistributed

In [2]:
def count_video(video_path):
  cap = cv2.VideoCapture(video_path)
  return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [3]:
def read_video(video_path, height, width):
  cap = cv2.VideoCapture(video_path)
  frames = []
  while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (width, height))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
  return np.array(frames, dtype='uint8'), len(frames)

In [4]:
dataset_path = "/content/drive/MyDrive/joon_del"
classes_name = os.listdir(dataset_path)
num_frames = []
for class_name in classes_name:
  videos_name = os.listdir(os.path.join(dataset_path, class_name))
  for video_name in videos_name:
    video_path = os.path.join(dataset_path, class_name, video_name)
    num_frame = count_video(video_path)
    num_frames.append(num_frame)
max_frames = np.max(num_frames)

In [5]:
print(max_frames)
height = 112
width = 112

316


In [6]:
del num_frames

In [7]:
X_frames = []
Y = []
for class_name in classes_name:
  videos_name = os.listdir(os.path.join(dataset_path, class_name))
  for video_name in videos_name:
    video_path = os.path.join(dataset_path, class_name, video_name)
    frames, num_frame = read_video(video_path, height, width)
    padded_frames = np.zeros(shape=(max_frames, height, width, 3))
    for i in range(num_frame):
      padded_frames[i] = frames[i] / 255
      # print(type(frames[i,0]))
    X_frames.append(padded_frames)
    Y.append(class_name)

In [8]:
X_frames = np.array(X_frames,dtype="float32")
Y = np.array(Y, dtype="int8")

In [9]:
print(X_frames.shape)
print(Y.shape)

(68, 316, 112, 112, 3)
(68,)


In [10]:
x_train,x_test,y_train,y_test = train_test_split(X_frames, Y,test_size=0.2)

In [11]:
del X_frames
del Y

In [12]:
model = tf.keras.models.Sequential([
            #CNN
            TimeDistributed(Conv2D(32,(3,3),activation='relu',input_shape=(None, height, width, 3))),
            # TimeDistributed(MaxPooling2D(pool_size=(2,2))),
            TimeDistributed(Conv2D(16,(3,3),strides=(2,2),activation='relu')),
            Dropout(0.4),
            TimeDistributed(Flatten()),

            #RNN
            SimpleRNN(128),
            Dense(2, activation='sigmoid')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [14]:
model.fit(x_train,y_train, batch_size=1, epochs=40)

Epoch 1/40
54/54 [==============================] - 51s 841ms/step - loss: 0.8032 - accuracy: 0.5000
Epoch 2/40
54/54 [==============================] - 45s 842ms/step - loss: 0.7632 - accuracy: 0.4630
Epoch 3/40
54/54 [==============================] - 43s 804ms/step - loss: 0.7466 - accuracy: 0.4074
Epoch 4/40
54/54 [==============================] - 42s 789ms/step - loss: 0.7205 - accuracy: 0.5370
Epoch 5/40
54/54 [==============================] - 43s 797ms/step - loss: 0.7530 - accuracy: 0.4259
Epoch 6/40
54/54 [==============================] - 44s 819ms/step - loss: 0.7352 - accuracy: 0.4815
Epoch 7/40
54/54 [==============================] - 42s 785ms/step - loss: 0.7380 - accuracy: 0.3889
Epoch 8/40
54/54 [==============================] - 41s 756ms/step - loss: 0.7956 - accuracy: 0.4815
Epoch 9/40
54/54 [==============================] - 43s 792ms/step - loss: 0.7204 - accuracy: 0.4074
Epoch 10/40
54/54 [==============================] - 43s 795ms/step - loss: 0.7080 - accura

In [17]:
del x_train
del y_train

In [18]:
del class_name
del dataset_path
del classes_name

In [19]:
del frames
del padded_frames

In [20]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 7s 7s/step - loss: 0.6529 - accuracy: 0.7143


[0.6528682708740234, 0.7142857313156128]

In [15]:
model.save_weights('/content/rnn_model.h5')

# Inference

In [21]:
def read_clip(video_path):
  frames = []
  cap = cv2.VideoCapture(video_path)
  while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (width, height))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
  return np.array(frames, dtype='uint8'), len(frames)

In [35]:
video_path = "/content/drive/MyDrive/joon_del/0/006.mp4"
frames, num_frame = read_clip(video_path)
padded_frames = np.zeros(shape=(max_frames, height, width, 3))
for i in range(num_frame):
  padded_frames[i] = frames[i] / 255
  # print(type(frames[i,0]))



In [36]:
padded_frames = np.array(padded_frames[np.newaxis,...])

In [28]:
padded_frames.shape

(1, 316, 112, 112, 3)

In [37]:
model.predict(padded_frames)

array([[0.35953975, 0.27919644]], dtype=float32)